In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, optimizers, Input, optimizers
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from time import sleep
from sklearn.preprocessing import StandardScaler
import time
import pprint
tf.__version__

In [ ]:
start = time.perf_counter()

In [ ]:
len(tf.config.list_physical_devices('GPU'))>0

In [ ]:
device_name = tf.test.gpu_device_name()
print(device_name)

In [ ]:
X_x_train = np.loadtxt("./data/X_x_train.txt")
X_y_train = np.loadtxt("./data/X_y_train.txt")
X_z_train = np.loadtxt("./data/X_z_train.txt")
y_train = np.loadtxt("./data/y_train.txt")

X_x_test = np.loadtxt("./data/X_x_train.txt")
X_y_test = np.loadtxt("./data/X_y_train.txt")
X_z_test = np.loadtxt("./data/X_z_train.txt")
y_test = np.loadtxt("./data/y_train.txt")

In [ ]:
# for i, r in tqdm(X_x_train[:], total=X_x_train.shape[0], desc="printing graphs..."):
#     print("plot" +str(i))
#     row = X_x_train[i,:-1]
#     y = X_x_train[i,-1:]
#     print(label_names[int(y)-1])
#     plt.plot(row)
#     plt.show()


In [ ]:
def standard_score_normalization(x):
    x_m = np.mean(x, axis=1).reshape(-1,1)
    x_std = np.std(x,axis=1).reshape(-1,1)
    return (x-x_m)/x_std

In [ ]:
norm = StandardScaler()

In [ ]:
X_x_train_norm = norm.fit_transform(X_x_train)
X_y_train_norm = norm.fit_transform(X_y_train)
X_z_train_norm = norm.fit_transform(X_y_train)

X_x_test_norm = norm.fit_transform(X_x_test)
X_y_test_norm = norm.fit_transform(X_y_test)
X_z_test_norm = norm.fit_transform(X_z_test)

### Below I'm comparing two different concatenation functions.

concat_xyz is vectorizing calculations while concat2_xyz is iterating

In [ ]:
def concat_xyz(x,y,z):
    data = np.empty((x.shape[0],x.shape[1],3))
#     x = np.expand_dims(x, axis=1) # same as x = x[:,np.newaxis,:]
#     y = np.expand_dims(y, axis=1)
#     z = np.expand_dims(z, axis=1)
    data[:,:,0] = x
    data[:,:,1] = y
    data[:,:,2] = z
    return data

In [ ]:
#speed test purposes
def concat2_xyz(x,y,z):
    data = np.empty((x.shape[0],x.shape[1],3))
    for i, rowx,rowy, rowz in zip(range(x.shape[0]),x,y,z):
        data[i,:,0] = rowx.reshape(1,-1)
        data[i,:,1] = rowy.reshape(1,-1)
        data[i,:,2] = rowz.reshape(1,-1)
    return data

In [ ]:
tic = time.perf_counter()
X_xyz_norm_train =  concat_xyz(X_x_train_norm,X_y_train_norm, X_z_train_norm)
X_xyz_norm_test = concat_xyz(X_x_test_norm,X_y_test_norm,X_z_test_norm)
toc = time.perf_counter()
print(f"concat_xyz in {toc - tic:0.4f} seconds")

In [ ]:
#speed test purposes
tic = time.perf_counter()
concat2_xyz(X_x_train_norm,X_y_train_norm, X_z_train_norm)
concat_xyz(X_x_test_norm,X_y_test_norm,X_z_test_norm)
toc = time.perf_counter()
print(f"concat2_xyz in {toc - tic:0.4f} seconds")

#### Vectorizing calculations is faster


In [ ]:
label_names = ['Walking', 'Walking upstairs', 'Walking downstairs', 'Sitting', 'Standing', 'Laying']
num_outputs = len(label_names)

In [ ]:
# one hot encoding

y_train = np.eye(len(label_names))[(y_train-1).astype(int)]
y_test = np.eye(len(label_names))[(y_train-1).astype(int)]

In [ ]:
X_xyz_norm_train.shape

In [ ]:
input_shape = X_xyz_norm_train.shape[1:]

In TF1 placeholder would be needed to get shape of `(None, 128, 3)` but here we take `(128, 3)` as input shape instead

<H1>Implementing CNN</H1>

In [ ]:
cnn = models.Sequential()
cnn.add(layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=input_shape))
# cnn.add(layers.MaxPool1D(pool_size=2))
cnn.add(layers.Flatten())
cnn.add(layers.Dense(100, activation='relu'))
cnn.add(layers.Dense(30, activation='relu'))
cnn.add(layers.Dense(num_outputs,activation='softmax'))

In [ ]:
cnn.summary()

In [ ]:
# optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
cnn.compile(optimizer="adam",
           loss="categorical_crossentropy",
           metrics="accuracy")

In [ ]:
cnnhistory = cnn.fit(X_xyz_norm_train, y_train, epochs=100, batch_size = 10)

In [ ]:
# cnn.save("my_model")
# tensorflow_graph = tf.saved_model.load("my_model")
# x = np.random.uniform(size=(4, 32)).astype(np.float32)
# predicted = tensorflow_graph(x).numpy()

In [ ]:
loss_values = cnnhistory.history['loss']
epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
acc_values = cnnhistory.history['accuracy']
epochs = range(1, len(loss_values)+1)

plt.plot(epochs, acc_values, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
y_test_pred = cnn.predict(X_xyz_norm_test)
pprint.pprint(y_test_pred)
pprint.pprint(y_test)

In [ ]:
# tf.math.confusion_matrix(y_test,yhat)

<H1>Implementing LSTM</H1>

In [ ]:
# LSTM

lstm = models.Sequential()
lstm.add(layers.Bidirectional(
    layers.LSTM(128),input_shape=input_shape
))

lstm.add(layers.Dropout(rate=0.2))
lstm.add(layers.Dense(6,activation="softmax"))



In [ ]:
lstm.summary()

In [ ]:
lstm.compile(optimizer = "adam",
            loss="categorical_crossentropy",
            metrics = "accuracy"
            )

In [ ]:
lstmhistory = lstm.fit(X_xyz_norm_train,y_train,epochs = 100,batch_size=10)

In [ ]:
loss_values = lstmhistory.history['loss']
epochs = range(1, len(loss_values)+1)

plt.plot(epochs, loss_values, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
acc_values = lstmhistory.history['accuracy']
epochs = range(1, len(loss_values)+1)

plt.plot(epochs, acc_values, label='Training Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
end = time.perf_counter()
print(f"notebook took {end - start:0.4f} seconds to finish")